In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('./data/1. 主数据集_25k/df_train_b1.csv')
test = pd.read_csv('./data/1. 主数据集_25k/df_test_b1.csv')
data = pd.concat([train, test]).reset_index(drop=True)

basic_b1 = pd.read_csv('./data/2. 细分数据集_25k/df_basic_b1.csv')
basic_b1.columns = ['cust_id', 'basic_1_other', 'basic_2_other', 'basic_3_other']

corp = pd.read_csv('./data/2. 细分数据集_25k/df_corp_b1.csv')

query = pd.read_csv('./data/2. 细分数据集_25k/df_query_b1.csv')
query.columns = ['cust_id', 'date_1_query', 'query_1_other', 'query_2_other', 'query_3_other',
                 'query_4_other', 'query_5_other', 'query_6_other']

loan2 = pd.read_csv('./data/2. 细分数据集_25k/df_loan2_b1.csv')
loan2.columns = ['cust_id', 'loan2_1_other', 'loan2_2_other', 'loan2_3_other', 'loan2_4_other', 'loan2_5_other']

data = pd.merge(data, basic_b1, on='cust_id', how='left')
data = pd.merge(data, corp, on='cust_id', how='left')
data = pd.merge(data, query, on='cust_id', how='left')
data = pd.merge(data, loan2, on='cust_id', how='left')

In [3]:
train = data[~data['label'].isna()].reset_index(drop=True)
train['adv_label'] = 0
test = data[data['label'].isna()].reset_index(drop=True)
test['adv_label'] = 1
no_train_fea = ['label', 'cust_id', 'adv_label']
features = [i for i in train.columns if i not in no_train_fea]

data = pd.concat([train, test]).reset_index(drop=True)
X = data[features]
y = data['adv_label']

In [21]:
data[features]

,basic_1,basic_2,basic_3,basic_4,basic_5,basic_6,basic_7,basic_8,basic_9,basic_10,...,query_2_other,query_3_other,query_4_other,query_5_other,query_6_other,loan2_1_other,loan2_2_other,loan2_3_other,loan2_4_other,loan2_5_other
0,-99,-99,-99.0,-99.0,1,2,37,0,0,2,...,2.0,1.0,0.0,1.0,0.0,10.0,12.0,633875.0,397090.0,320365.0
1,-99,-99,-99.0,-99.0,1,2,49,0,0,2,...,2.0,1.0,0.0,1.0,0.0,7.0,19.0,352000.0,177810.0,185593.0
2,-99,-99,-99.0,-99.0,1,1,52,0,0,3,...,2.0,1.0,0.0,1.0,0.0,2.0,5.0,82000.0,57264.0,56618.0
3,-99,-99,-99.0,-99.0,1,1,54,0,0,5,...,2.0,0.0,0.0,0.0,0.0,8.0,21.0,97800.0,81408.0,77134.0
4,-99,-99,-99.0,-99.0,2,3,33,0,0,3,...,2.0,0.0,0.0,0.0,0.0,6.0,21.0,185750.0,172493.0,179053.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,-99,-99,-99.0,-99.0,2,2,34,0,0,1,...,8.0,0.0,0.0,0.0,0.0,2.0,13.0,26000.0,14848.0,11585.0
24996,1,111,886.0,886.0,1,1,44,0,0,5,...,2.0,1.0,0.0,1.0,0.0,5.0,7.0,286000.0,51039.0,66590.0
24997,-99,-99,-99.0,-99.0,1,1,25,0,0,3,...,8.0,0.0,0.0,0.0,0.0,2.0,2.0,85367.0,3905.0,3970.0
24998,-99,-99,-99.0,-99.0,1,1,54,0,0,3,...,2.0,2.0,0.0,2.0,0.0,1.0,1.0,15000.0,0.0,1666.0


In [23]:
y.iloc[1]

0

In [27]:
# 定义模型参数
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, classification_report

feat_imp_df = pd.DataFrame({'feat': features, 'imp': 0})
KF = StratifiedKFold(n_splits=5, random_state=2021, shuffle=True)


params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'n_jobs': 30,
    'learning_rate': 0.1,
    'num_leaves': 16,
    'max_depth': 4,
    'tree_learner': 'serial',
    'colsample_bytree': 0.85,
    'subsample_freq': 1,
    'subsample': 0.8,
    'num_boost_round': 5000,
    'max_bin': 255,
    'verbose': -1,
    'seed': 2021,
    'bagging_seed': 2021,
    'feature_fraction_seed': 2021,
    'early_stopping_rounds': 100,
}
# 训练集标签预测 df 初始化
oof_lgb = np.zeros(len(data))


# 模型训练
for fold_, (trn_idx, val_idx) in enumerate(KF.split(X.values, y.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(data.iloc[trn_idx][features], label=y.iloc[trn_idx])
    val_data = lgb.Dataset(data.iloc[val_idx][features], label=y.iloc[val_idx])
    num_round = 3000
    clf = lgb.train(
        params,
        trn_data,
        num_round,
        valid_sets=[trn_data, val_data],
        verbose_eval=100,
        early_stopping_rounds=20,
    )

    oof_lgb[val_idx] = clf.predict(data.iloc[val_idx][features], num_iteration=clf.best_iteration)
    #predictions_lgb[:] += clf.predict(test[features], num_iteration=clf.best_iteration) / 5
    feat_imp_df['imp'] += clf.feature_importance() / 5
# 计算 metrics
print("AUC score: {}".format(roc_auc_score(y, oof_lgb)))
print("F1 score: {}".format(f1_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Precision score: {}".format(precision_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Recall score: {}".format(recall_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))

fold n°0
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.996688	valid_1's auc: 0.991136
[200]	training's auc: 0.998869	valid_1's auc: 0.991443
Early stopping, best iteration is:
[187]	training's auc: 0.998704	valid_1's auc: 0.991515
fold n°1
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.996338	valid_1's auc: 0.993934
[200]	training's auc: 0.998686	valid_1's auc: 0.994231
[300]	training's auc: 0.999528	valid_1's auc: 0.994211
Early stopping, best iteration is:
[244]	training's auc: 0.999124	valid_1's auc: 0.994333
fold n°2
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.996359	valid_1's auc: 0.993982
Early stopping, best iteration is:
[84]	training's auc: 0.995698	valid_1's auc: 0.994013
fold n°3
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.996409	valid_1's auc: 0.993672
[200]	training's auc: 0.998705	valid_1's auc: 0.99365
Early st

In [30]:
feat_imp_df.sort_values(by='imp')

,feat,imp
67,overdue_8,0.0
65,overdue_6,0.0
23,loan1_9,0.0
22,loan1_8,0.0
60,overdue_1,0.0
...,...,...
98,query_10,76.8
92,query_4,77.4
104,industry,77.6
105,scope,111.4
